<a href="https://colab.research.google.com/github/moaaz12-web/Q-A-on-pdf-files/blob/main/Custom_chatbot_using_langchain_and_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALL DEPENDENCIES

In [ ]:
!pip install llama-index==0.5.0
!pip install langchain
! pip install pypdf2

# CONVERT PDF TO TEXT FILE

In [ ]:
import PyPDF2
import json

# Open the PDF file
pdf_file = open('/content/FB-23-18 SPECIFICATIONS.pdf', 'rb')

# Create a PDF reader object
pdf_reader = PyPDF2.PdfReader(pdf_file)

# Create an empty dictionary to store the PDF data
pdf_data = {}

# Loop over each page in the PDF file
for page_num in range(len(reader.pages)):
  # Get the current page object
  page = pdf_reader.pages[page_num]

  # Get the text content of the page
  page_text = page.extract_text()

  # Add the page content to the dictionary
  pdf_data[f'page_{page_num+1}'] = page_text

  # Convert the dictionary to JSON format
  json_data = json.dumps(pdf_data)

# Print the JSON data
print(json_data)

In [ ]:
# import os module
import os

# create a new file
filename = "pdf_data.txt"
file = open(filename, "w")

# write content to the file
for i in range(1, len(pdf_data.keys())+1):
    file.write(pdf_data[f"page_{i}"])
    file.write("\n") # add a newline character after each page

# close the file
file.close()

# print a message to confirm the file creation
print(f"{filename} has been created!")


pdf_data.txt has been created!


In [ ]:

from langchain import OpenAI
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex,GPTSimpleVectorIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
import sys
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = 'OPENAI_API_KEY'

# CREATE VECTOR INDEX OF TEXT FILE

In [ ]:
def construct_index(directory_path):
  # set maximum input size
  max_input_size = 4096
  # set number of output tokens
  num_outputs = 256
  # set maximum chunk overlap
  max_chunk_overlap = 20
  # set chunk size limit
  chunk_size_limit = 600

  prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

  # define LLM
  llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_outputs))
  
  documents = SimpleDirectoryReader(directory_path).load_data()
  
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)
  index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
  
  index.save_to_disk('index.json')
  
  return index

In [ ]:
index = construct_index("/content")

# ASK QUESTIONS FROM YOUR PDF

In [ ]:
question="""
Explain Galvanized Ferrous Metal Substrates
"""

In [ ]:
response = index.query(question)

In [ ]:
response.response

'\n\nGalvanized ferrous metal substrates are metal surfaces that have been treated with a protective coating of zinc to prevent corrosion. The coating is applied by dipping the metal in a bath of molten zinc or by electroplating. The coating provides a barrier between the metal and the environment, protecting it from rust and corrosion. The coating also provides a decorative finish that can be used to enhance the appearance of the metal. It is important to ensure that the surface alkalinity is within the range recommended by the paint manufacturer and that the moisture content of the substrate is not greater than the maximum values recommended for the specific substrate material. The surface should also be cleaned with a detergent solution and a bristle brush to ensure that all soil is thoroughly dislodged and can be removed by rinsing.'